In [1]:
import imagej
import pandas as pd

To create a new class that takes a net.imagej.ImageJ gateway instance and adds new methods, you can create a wrapper class in Python. This class will accept an ImageJ instance and provide additional methods to enhance its functionality.

In [2]:
class ImageJWrapper:
    """
    A wrapper class for the ImageJ gateway instance, providing additional convenience methods
    for common image processing tasks and interactions with the ImageJ UI.

    Attributes:
        ij (net.imagej.ImageJ): The ImageJ gateway instance.
    """
    
    def __init__(self, ij_instance):
        """
        Initializes the ImageJWrapper with an ImageJ instance.
    
        Args:
            ij_instance (net.imagej.ImageJ): The ImageJ gateway instance to be wrapped.
        """
        self.ij = ij_instance

    def open_czi_serie(self, image_filename, serie_numbers):
        """
        Opens a CZI file in ImageJ and imports specified series using a macro.
    
        Args:
            image_filename (str): The path to the CZI file to be opened.
            serie_numbers (str): A comma-separated string of series numbers to be imported.
        """

        # Run [ImageJ macro](https://py.imagej.net/en/latest/07-Running-Macros-Scripts-and-Plugins.html#using-imagej-macros-ij-py-run-macro)
        macro = """
#@ String imageFile
#@ String serie_n

run("Bio-Formats Importer", 
    "open=[" + imageFile + "] autoscale color_mode=Composite rois_import=[ROI manager] view=Hyperstack stack_order=XYCZT " + addSerieText(serie_n));

function addSerieText(serieString) {
    // Converted array
    separatedArray = split(serieString,", ");
    
    serieArray = separatedArray;
    for(var i=0;i<separatedArray.length;i++){
        serieArray[i]="series_"+separatedArray[i];
    }
    
    return String.join(separatedArray, ' ')   	
}
        
        """
        
        args = {
            'imageFile': image_filename,
            'serie_n': serie_numbers
        }
        
        self.ij.py.run_macro(macro, args)

    def get_all_open_image_titles(self):
        """
        Retrieves the titles of all currently open images in ImageJ.
    
        Returns:
            List[str]: A list of the titles of all open images in ImageJ.
        """
        # Now let's get all open ImageJ images
        WindowManager = self.ij.WindowManager
        image_titles = WindowManager.getImageTitles()
        
        return list(image_titles)

    def get_image_by_title(self, title):
        """
        Selects an ImageJ image by its title.

        Args:
            title (str): The title of the ImageJ image to be selected.

        Returns:
            ImagePlus: The ImagePlus object representing the selected image, or None if not found.
        """
        WindowManager = self.ij.WindowManager        
        return WindowManager.getImage(title)

    def set_current_image_by_title(self, title):
        """
        Set a the current ImageJ image by its title.

        Args:
            title (str): The title of the ImageJ image to be selected.

        Returns:
            None
        """
        
        WindowManager = self.ij.WindowManager    

        imp = self.get_image_by_title(title)

        WindowManager.setTempCurrentImage(imp)

    def close_all(self):
        """
        Close all open images
        """
        self.ij.IJ.run("Close All")

In [ ]:
# Initialize ImageJ from a local installation
ij = imagej.init('/Fiji.app', mode='interactive')

ij.ui().showUI() # if you want to display the GUI immediately

# Verify the version of ImageJ
print(ij.getVersion())

# Get ImageJ Wrapper instance
ij_wrapper = ImageJWrapper(ij)

In [4]:
ij_wrapper.close_all()

In [5]:
def imagej_workflow(ij_wrapper, image_filename, serie_numbers):
    
    ij_wrapper.open_czi_serie(image_filename, serie_numbers)

    for t in ij_wrapper.get_all_open_image_titles():
        
        ij_wrapper.set_current_image_by_title(t)
        
        # convert image to RGB type
        ij.IJ.run("RGB Color")
    
        # Add scalebar (OPTIONAL)
        scalebar_options = "width=15 height=15 thickness=6 font=30 color=Black background=None location=[Lower Right] bold"
        ij.IJ.run("Scale Bar...", scalebar_options)
        
        # Export image
        output_image = str(t) + ".png"
        ij.IJ.saveAs('PNG', output_image)
    
    ij_wrapper.close_all()

In [ ]:
load_info = pd.read_csv('load_series_info.csv', quotechar='"', skipinitialspace=True)
load_info

In [ ]:
for _, row in load_info.iterrows():

    print("---\nRunning image: " + row.file_path)
    imagej_workflow(ij_wrapper, row.file_path, row.serieString)
    print("Done!\n---\n")